# Firework Algorithm for Optimization

- Autores originais: [*Ying Tan*](https://scholar.google.com/citations?user=PjNxSPsAAAAJ&hl=en) e [*Yuanchun Zhu*](https://ieeexplore.ieee.org/author/38008573100)
- Ano de Publicação: 2010
- Editora: [Springer](https://link.springer.com/chapter/10.1007/978-3-642-13495-1_44)
- Conferência: [Advances in Swarm Intelligence](https://link.springer.com/book/10.1007/978-3-642-13495-1): First International Conference, ICSI 2010, Beijing, China, June 12-15, 2010, Proceedings, Part I 1

## Descrição:
FA é um algoritmo de inteligência de enxames inspirado na explosão de fogos de artifício. Esse algoritmo é utilizado para a otimização global de funções complexas. São fornecidos dois processos de *“explosão”* (busca) e os mecanismos para manter a diversidade das *“faíscas”*.

In [70]:
import numpy as np
import pandas as pd
import random

## Funções Objetivos

In [71]:
def sphere_function(x):
    result = x**2
    return np.sum(result)

In [72]:
def rastrigin(x):
    n = len(x)
    total = 0
    total = x**2 - 10*np.cos(2*np.pi*x)
    return (10*n) + total

In [73]:
def rosenbrook(x):
    n = len(x)
    term1 = 100 * (x[1:] - x[:-1]**2)**2
    temr2 = (x[1:] -1)**2
    sumterm = np.sum(term1 + term2)
    return sumterm

In [74]:
def ackley(x):
    n = len(x)
    sum1 = np.sum(x*2)
    sum2 = np.sum(np.cos(2 * np.pi * x**2))
    term1 = -20 * np.exp(-0.2 * np.sqrt(1/n * sum1))
    term2 = -np.exp(1/n * sum2)
    return term1 + term2 + 20 + np.exp(1)

### Distribuição Gaussiana:

In [75]:
value = random.gauss(1,1)
value

0.5168717843486446

### Distribuição Uniforme:

In [76]:
value = random.uniform(-1,1)
value

0.09789054563351529

## Inicializando posições dos Fogos de Artifício:

In [77]:
def initialize_firework_locations(n, d, xmin, xmax):
    fireworks = xmin + (xmax - xmin) * np.random.rand(n,d)
    return fireworks

## Calculando o número de faíscas:

In [78]:
def sparks_number(fx, fX, m, ymax, eps=.00001):
    sum_difference_of_worst_fit_and_fx = np.sum(ymax-fX) + eps
    difference_of_max_and_fx = (ymax - fx + eps)
    s = m * difference_of_max_and_fx/sum_difference_of_worst_fit_and_fx
    s = firework_boundaries(0.04, 0.8, m, s)
    return s

In [79]:
def firework_boundaries(a, b, m, s):
    term1 = np.round(a*m) * (s < (a*m))
    term2 = np.round(b*m) * (s> b*m)
    term3 = np.round(s) * (~(s < (a*m)) & ~((s > b *m) & (a<b<1)))
    s_new = term1 + term2 + term3
    return s_new

## Obtendo localização das Faíscas:

### Seletor de dimensões aleatório

In [80]:
def random_dimension_selector(d):
    xj = np.zeros(d)
    z = round(d * random.uniform(0,1))
    random_dimensions = np.random.choice(d, z, replace=False)
    for i in random_dimensions:
        xj[i] = 1
    return xj
    

### Amplitude de explosão

In [81]:
def explosion_amplitude(A_hat, fx,fX, ymin, eps=.00001):
    sum_difference_of_fx_and_best_fit = np.sum(fX - ymin) + eps
    difference_of_fx_and_min = (fx - ymin + eps)
    A = A_hat * difference_of_fx_and_min/sum_difference_of_fx_and_best_fit
    return A

### Obtendo faísca: Estratégia da Explosão

In [82]:
def obtain_location_of_spark(d, amp, xmin, xmax, xi):
    #selecting z dimensions to be affected
    xj = xi
    random_selected_positions = random_dimension_selector(d) 
    h = amp * random.uniform(-1 ,1) 
    random_affected_dimensions = xj + (random_selected_positions* h)
    #mapping affected dimensions to potential space
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

### Mapeamento para espaço potencial:

In [83]:
def map_to_potential_space(xmin, xmax, affected):
    inbound = xmin + abs(affected)%(xmax - xmin)
    term1 = (affected > xmax) * inbound
    term2 =  (affected < xmin) * inbound
    fixed_bounds = term1 + term2
    mapping = (fixed_bounds == 0) * affected + fixed_bounds 
    return mapping

In [84]:
xi = np.random.rand(3)
xmin = np.array([-3.12, -4.5, -6.6])
xmax = np.array([3.12, 4.5, 6.6])
d = 3
amp = 5
obtain_location_of_spark(d, amp, xmin, xmax, xi)

array([0.63037814, 0.79600478, 0.56539568])

### Obtendo faísca: Estratégia de Faíscas Gaussianas:

In [85]:
def gaussian_spark_location(xmin, xmax, d, xi):
    xj = xi
    random_dimensions_selected = random_dimension_selector(d)
    random_affected_dimensions = (xj * np.logical_not(random_dimensions_selected)) + xj * (random_dimensions_selected * np.random.normal(1,1,d))
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

In [86]:
d =3
np.random.normal(1,1,d)

array([2.84598457, 0.39379428, 0.58864958])

In [87]:
np.random.rand(3)

array([0.19212495, 0.07873747, 0.71101   ])

## Seleção de Localizações:

### Calculando Distância Geral:

In [88]:
def general_distance(xi , X, d):
    sum_of_distances = np.sum(abs(xi - X))
    return sum_of_distances

### Calculando probabilidade de seleção:

In [89]:
def location_probability(RX, Rxi,d):
    pxi = Rxi/RX
    return pxi

## Framework ALgoritmo de Otimização Fogos de Artifício:

In [ ]:
#localizações
n = 5
#quantidade de faíscas
m = 50
#dimensões
d = 10
#parâmetro para amplitude
a = 40
#faíscas gaussianas
gauss = 15

#Número de fogos na estratégia gaussiana
m_hat = 5

#minimo valor do espaço potencial
xmin = -20

#máximo valor do espaço potencial
xmax = 20

#Inicializando fogoso de artifício
X = initialize_firework_locations(n, d, xmin, xmax)

#Total de iterações
explosion_generations = 1000

current_generation = 0
while(current_generation < explosion_generations):
        #f(X)
    y = np.zeros((n,1))
    for i in range(n):
        y[i] = sphere_function(X[i])

    #best fit
    ymin = y.min()
    #worst fit
    ymax = y.max()

    #Numero de faíscas por fogo de artifício
    S = np.zeros((n,1))
    #Amplitude de cada fogo de artificio
    A = np.zeros((n,1))

    #Seleção de quantidade de faíscas e Amplitude para cada Fogo de artifício
    for i in range(n):
        S[i] = sparks_number(y[i],y, m, ymax) 
        A[i] = explosion_amplitude(a, y[i],y, ymin)

    total_sparks = np.sum(S)
    new_locations = X
    generation = np.array((n,d))
    for i in range(n):
        xi = X[i]
        n_sparks = int(S[i][0])
        sparks = np.zeros((n_sparks,d))
        for j in range(n_sparks):
            sparks[j] = obtain_location_of_spark(d, a, xmin, xmax, xi)
        new_locations = np.vstack((new_locations, sparks)) 

    random_fireworks = np.sort(np.random.choice(n,m_hat, replace=False))
    for i in range(m_hat):
        xi = X[random_fireworks[i]]
        gaussian_sparks = np.zeros((gauss,d))
        for j in range(gauss):
            gaussian_sparks[j] = gaussian_spark_location(xmin, xmax, d, xi)
        new_locations = np.vstack((new_locations, gaussian_sparks)) 
    total_locations = int(total_sparks) + n + m_hat*gauss
    fx_new_locations = np.zeros((total_locations, 1))
    for i in range(total_locations):
        fx_new_locations[i] = sphere_function(new_locations[i])

    current_best_location = fx_new_locations.argmin()
    best_location = new_locations[current_best_location]
    new_locations = np.delete(new_locations, current_best_location,0)
    RX = np.zeros((total_locations -1, 1)) 
    for i in range(total_locations-1):
        RX[i] = general_distance(new_locations[i], new_locations, d)

    prob = np.zeros((total_locations, 1))
    sum_RX = np.sum(RX)
    for i in range(total_locations-1):
        prob[i] = location_probability(sum_RX, RX[i],d)
    prob = prob.flatten()
    indexes = np.arange(total_locations)

    next_generation_selection = np.random.choice(indexes, n -1, p=prob, replace=False)

    X = np.zeros((n-1,d))
    for i in range(n-1):
        X[i] = next_generation_selection[i]
    X = np.vstack((best_location, X))
    print("current best location:\n",best_location)
    print("FX of best location: \n", fx_new_locations[current_best_location])
    print("current iteration: ", current_generation)
    current_generation+=1

current best location:
 [  1.26102813  -8.13561396  -3.23053031   0.57236728   7.13943642
  -7.41017159 -13.88916937  -1.00565111   1.15999972   7.74953447]
FX of best location: 
 [439.74577595]
current iteration:  0
current best location:
 [ 0.63570956 -5.40989123 -3.23053031  0.57283738  3.89420786  9.75355173
  1.62112434  0.79858509  2.2341675   1.78903414]
FX of best location: 
 [162.19007457]
current iteration:  1
current best location:
 [-0.67483874  2.         -0.67483874 -0.67483874 -0.67483874 -0.67483874
 -0.67483874 -0.67483874 -0.67483874 -0.67483874]
FX of best location: 
 [8.09866598]
current iteration:  2
current best location:
 [-0.49770702  0.31706129 -0.6390655  -0.83804224 -0.53805554 -0.66530183
 -0.67483874  0.52177027 -1.30520739 -0.19545728]
FX of best location: 
 [4.66051139]
current iteration:  3
current best location:
 [-0.49770702  0.31706129 -0.6390655  -0.83804224 -0.74384386 -0.66530183
 -0.67483874  0.58178719  0.11902891 -0.19545728]
FX of best location

<iframe style="border-radius:12px" src="https://open.spotify.com/embed/track/3GFiNXCDyjCw14IRQIQszu?utm_source=generator" width="100%" height="352" frameBorder="0" allowfullscreen="" allow="autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture" loading="lazy"></iframe>
